In [1]:
import pandas as pd
import numpy as np

from datetime import datetime as dt

import yfinance as yf
from yahooquery import Ticker

from merlion.models.forecast.arima import Arima, ArimaConfig
from merlion.models.forecast.prophet import Prophet, ProphetConfig
from merlion.models.forecast.smoother import MSES, MSESConfig
from merlion.utils.time_series import TimeSeries
from merlion.transform.base import Identity
from merlion.transform.resample import TemporalResample

from causalimpact import CausalImpact
import tensorflow as tf
import os

from sklearn.metrics import mean_squared_error, r2_score

import plotly.express as px
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

c:\Users\Reuel Group\anaconda3\envs\causal\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## can use yfinance or yahooquery to pull stock price data into pandas
df_crm = yf.Ticker("CRM").history(start='2022-12-06', end='2023-01-05', interval="1h")
df_crm.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2022-12-06 09:30:00-05:00,132.710007,133.770004,130.910004,132.509995,5316748,0.0,0.0
2022-12-06 10:30:00-05:00,132.509995,133.729904,132.149994,133.199997,2172621,0.0,0.0
2022-12-06 11:30:00-05:00,133.190002,134.550003,132.649994,132.919998,1705820,0.0,0.0
2022-12-06 12:30:00-05:00,132.910004,133.091995,132.235001,132.279999,1303352,0.0,0.0
2022-12-06 13:30:00-05:00,132.279999,133.300003,131.979996,132.725006,1709159,0.0,0.0


In [3]:
## Companies that share similar characteristics that could be useful for the analysis
basket = ['CRM','MSFT','ADBE','ORCL','NOW','WDAY','HUBS','MNDY','TEAM','SNOW']

In [4]:
df_stocks = pd.DataFrame([],columns=['Name','Ticker','Price','Datetime'])

for i, j in enumerate(basket):
    ## get hourly price data
    temp_df = yf.Ticker(j).history(start='2022-12-06', end='2023-01-05', interval="1h")
    temp_df.reset_index(drop=False,inplace=True)
    temp_df.loc[:,'Ticker'] = j
    temp_df.rename(columns={'Open':'Price'},inplace=True)
    temp_df.loc[:,'Name'] = yf.Ticker(j).info['longName']
    temp_df = temp_df[['Name','Ticker','Price','Datetime']]
    df_stocks = pd.concat([df_stocks,temp_df])

df_stocks.head()

C:\Users\Reuel Group\AppData\Local\Temp\ipykernel_23588\962575659.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_stocks = pd.concat([df_stocks,temp_df])


,Name,Ticker,Price,Datetime
0,"Salesforce, Inc.",CRM,132.710007,2022-12-06 09:30:00-05:00
1,"Salesforce, Inc.",CRM,132.509995,2022-12-06 10:30:00-05:00
2,"Salesforce, Inc.",CRM,133.190002,2022-12-06 11:30:00-05:00
3,"Salesforce, Inc.",CRM,132.910004,2022-12-06 12:30:00-05:00
4,"Salesforce, Inc.",CRM,132.279999,2022-12-06 13:30:00-05:00


In [5]:
df_stocks['Ticker'].unique()

array(['CRM', 'MSFT', 'ADBE', 'ORCL', 'NOW', 'WDAY', 'HUBS', 'MNDY',
       'TEAM', 'SNOW'], dtype=object)

In [6]:
df_stocks['Datetime'] = pd.to_datetime(df_stocks['Datetime'])

In [7]:
df_stocks['Date'] = df_stocks['Datetime'].dt.strftime('%Y-%m-%d')
df_stocks['Timeseries_Index'] = df_stocks.groupby(['Ticker'],as_index=False).cumcount() + 1
df_stocks['Day_Num'] = df_stocks['Date'].rank(method='dense')
df_stocks['Hour_Num'] = df_stocks.groupby(['Ticker','Date'],as_index=False).cumcount() + 1

df_stocks['Datetime'] = df_stocks['Datetime'].dt.tz_localize(None)

In [8]:
df_stocks.head()

,Name,Ticker,Price,Datetime,Date,Timeseries_Index,Day_Num,Hour_Num
0,"Salesforce, Inc.",CRM,132.710007,2022-12-06 09:30:00,2022-12-06,1,1.0,1
1,"Salesforce, Inc.",CRM,132.509995,2022-12-06 10:30:00,2022-12-06,2,1.0,2
2,"Salesforce, Inc.",CRM,133.190002,2022-12-06 11:30:00,2022-12-06,3,1.0,3
3,"Salesforce, Inc.",CRM,132.910004,2022-12-06 12:30:00,2022-12-06,4,1.0,4
4,"Salesforce, Inc.",CRM,132.279999,2022-12-06 13:30:00,2022-12-06,5,1.0,5


In [9]:
fig_df = df_stocks[df_stocks['Ticker'].isin(['CRM'])]
fig_df['Layoff Announced'] = fig_df['Day_Num']==20
fig = px.line(fig_df
              ,x='Timeseries_Index'
              ,y='Price'
              ,markers=True
              ,color='Layoff Announced'
              ,hover_data=['Datetime','Timeseries_Index','Price','Layoff Announced']
              ,title='Stock price for {} ({})'.format(fig_df['Name'].unique()[0],fig_df['Ticker'].unique()[0]))

fig.update_layout(shapes=
                  [dict(type= 'line',
                        yref= 'paper', y0= 0, y1= 1,
                        xref= 'x', x0='133', x1='133',
                        line=dict(color="#959696",
                                  width=2,
                                  dash="dot"))]
                 ,plot_bgcolor='#f5f6f7')
fig.show()

C:\Users\Reuel Group\AppData\Local\Temp\ipykernel_23588\867132878.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fig_df['Layoff Announced'] = fig_df['Day_Num']==20


In [12]:
## prior day last price vs post day first price
#prior day last price
pdlp = df_stocks[(df_stocks['Ticker']=='CRM') & (df_stocks['Timeseries_Index']==133)]['Price'].values[0]

#post day first price
pdfp = df_stocks[(df_stocks['Ticker']=='CRM') & (df_stocks['Timeseries_Index']==134)]['Price'].values[0]

pre_post1 = round((pdfp-pdlp)/pdlp,4)
print('Pre-Post last price vs first price: {:.2%}'.format(pre_post1))

## prior day average vs post day average

prda = df_stocks[(df_stocks['Ticker']=='CRM') & (df_stocks['Date']=='2023-01-03')]['Price'].mean()

#post day first price
poda = df_stocks[(df_stocks['Ticker']=='CRM') & (df_stocks['Date']=='2023-01-04')]['Price'].mean()

pre_post1 = round((poda-prda)/prda,4)
print('Pre-Post prior day average price vs post day average price: {:.2%}'.format(pre_post1))

Pre-Post last price vs first price: 4.55%
Pre-Post prior day average price vs post day average price: 3.70%


In [11]:
#CRM + basket mean
basket_stocks = df_stocks2[df_stocks2['Ticker']!='CRM']
basket_stocks = basket_stocks.groupby(['Timeseries_Index']).agg(
    Relative_Price=('Relative_Price',np.mean)
)
basket_stocks.reset_index(drop=False,inplace=True)
basket_stocks['Ticker'] = 'BASKET'
crm_df = df_stocks2[df_stocks2['Ticker']=='CRM']
crm_df = crm_df[['Timeseries_Index','Relative_Price','Ticker']]
fig_df = pd.concat([crm_df,basket_stocks])
fig = px.line(fig_df
              ,x='Timeseries_Index'
              ,y='Relative_Price'
              ,markers=True
              ,color='Ticker'
              ,title='CRM vs Basket of Similar Stocks')

fig.update_layout(shapes=
                  [dict(type= 'line',
                        yref= 'paper', y0= 0, y1= 1,
                        xref= 'x', x0='133', x1='133',
                        line=dict(color="#959696",
                                  width=2,
                                  dash="dot"))]
                 ,plot_bgcolor='#f5f6f7')
fig.show()

C:\Users\Reuel Group\AppData\Local\Temp\ipykernel_23588\2484785428.py:3: FutureWarning:

The provided callable <function mean at 0x0000021BC0F4FE20> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.

